# Paper roll cut - Modeling

## Autores:
- Alejandro De Haro
- Daniel Escobosa
- Pablo Berastegui
- Jose María Benitez


### Importing packages and modules


In [23]:
import pyomo.environ as pe
import pyomo.opt as po

### Create the model

### Order to build the model:
1. Sets  
2. Parameters  
3. Variables  
4. Objective function  
5. Constraints  
6. Solver definition and solve statement  
7. Results

In [24]:
model = pe.ConcreteModel()

### Sets

- $c \in \{1,2,3,4,5,6,7,8,9,10\}$: **option** orders.  
- n :  master paper rolls.

In [34]:
model.c = pe.Set(initialize=[i for i in range(1,11)])

N = 200    # Fixed number of master rolls
model.n = pe.Set(initialize=[i for i in range(1,N)])

'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


### Parameters

In [26]:
n_rolls = {
    1: 20,
    2: 30,
    3: 15,
    4: 25,
    5: 40,
    6: 25,
    7: 50,
    8: 15,
    9: 10,
    10: 5
}
lenght_order = {
    1: 0.5,
    2: 0.75,
    3: 1,
    4: 1.25,
    5: 1.5,
    6: 1.75,
    7: 2,
    8: 2.25,
    9: 2.5,
    10: 2.75
}
model.R = pe.Param(model.c, initialize=n_rolls)


model.L = pe.Param(model.c, initialize=lenght_order)

model.QL = pe.Param(initialize=3)
model.QP = pe.Param(initialize=5)  

### Variables

- $y_{n} \in \{0,1\}$: use of master paper roll n
- $z_{c,n} \in \{0,5\}$: number of orders c in paper roll n
- $d_{n} \in \{0,1\}$: order type 1 in roll n


In [27]:
model.y = pe.Var(model.n, within=pe.Binary)
model.z = pe.Var(model.c, model.n, within=pe.NonNegativeIntegers)
model.d = pe.Var(model.n, within=pe.Binary)

### Objective Function

We **minimize total number of rolls** of the chosen options:

In [28]:
def obj_rule(model):
    return sum(model.y[n] for n in model.n)
model.min_waste= pe.Objective(rule=obj_rule, sense=pe.minimize)

### Constraints

In [ ]:
# 1) Maximum length of master paper roll n [m]
#    sum_c L_c * z_{c,n} + l_n = QL     ∀ n
def max_length_rule(model, n):
    return sum(model.L[c] * model.z[c, n] for c in model.c) <= model.QL * model.y[n]
model.max_length = pe.Constraint(model.n, rule=max_length_rule)

# 2) Maximum number of cuts in master paper roll n [units]
#    sum_c z_{c,n} ≤ QP                  ∀ n
def max_cuts_rule(model, n):
    return sum(model.z[c, n] for c in model.c) <= model.QP * model.y[n]
model.max_cuts = pe.Constraint(model.n, rule=max_cuts_rule)

# 3) Meet clients’ order c [units]
#    sum_n z_{c,n} = R_c                 ∀ c
def meet_orders_rule(model, c):
    return sum(model.z[c, n] for n in model.n) == model.R[c]
model.meet_orders = pe.Constraint(model.c, rule=meet_orders_rule)

# 4) Order type 1 with order type 5 or type 9
#    z_{1,n} ≤ 3 · d_n                    ∀ n
#    z_{5,n} + z_{9,n} ≥ d_n              ∀ n
def type1_limit_rule(model, n):
    return model.z[1, n] <= 3 * model.d[n]
model.type1_limit = pe.Constraint(model.n, rule=type1_limit_rule)

def pair_with_5_or_9_rule(model, n):
    # Guard against missing types 5 or 9 in the set (if the set is ever changed)
    z5 = model.z[5, n] if 5 in list(model.c) else 0
    z9 = model.z[9, n] if 9 in list(model.c) else 0
    return z5 + z9 >= model.d[n]
model.pair_with_5_or_9 = pe.Constraint(model.n, rule=pair_with_5_or_9_rule)

# 5) Positive waste in paper roll n [m]
#    l_n ≥ 0  ∀ n  (already enforced by the variable domain NonNegativeReals)

'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


### Solver definition and solve statement

In [31]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Set parameter Username
Set parameter LicenseID to value 2704768
Academic license - for non-commercial use only - expires 2026-09-08
Read LP format model from file /var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/tmpew01qew8.pyomo.lp
Reading time = 0.00 seconds
x1: 806 rows, 2388 columns, 7363 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A354)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 806 rows, 2388 columns and 7363 nonzeros
Model fingerprint: 0x52cd565f
Variable types: 0 continuous, 2388 integer (398 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 5e+01]
Presolve time: 0.01s
Presolved: 806 rows, 2388 columns, 7363 nonzeros
Variable types: 0 continuous, 2388 integer (1393 binary)

Root relaxation: objective 1.187500e+02, 1386 iterations, 0.01 seconds (0.01 

### Results

We report:
- **Optimal total number of rolls** .

In [41]:
print(f"Numero optimo de rollos: {pe.value(model.min_waste)}")
rolls_check=0
for j in model.n:
    if pe.value(model.y[j]) > 0.5:  # si el rollo j se usa
        print(f"Rollo maestro {j}:")
        for i in model.c:
            qty = pe.value(model.z[i,j])
            if qty > 0:
                print(f"  - {qty} cortes de longitud {pe.value(model.L[i])}")
        rolls_check+=1
print(f"Verificamos el numero total de rollos para ver que cuadre: {rolls_check}")

Numero optimo de rollos: 127.0
Rollo maestro 1:
  - 2.0 cortes de longitud 1.5
Rollo maestro 2:
  - 3.0 cortes de longitud 0.5
  - 1.0 cortes de longitud 1.5
Rollo maestro 4:
  - 2.0 cortes de longitud 1.5
Rollo maestro 5:
  - 1.0 cortes de longitud 1
  - 1.0 cortes de longitud 2
Rollo maestro 6:
  - 1.0 cortes de longitud 2.5
Rollo maestro 7:
  - 2.0 cortes de longitud 1.5
Rollo maestro 8:
  - 1.0 cortes de longitud 1.25
  - 1.0 cortes de longitud 1.75
Rollo maestro 9:
  - 1.0 cortes de longitud 2.25
Rollo maestro 10:
  - 1.0 cortes de longitud 0.75
  - 1.0 cortes de longitud 2.25
Rollo maestro 13:
  - 1.0 cortes de longitud 0.75
  - 1.0 cortes de longitud 2.25
Rollo maestro 16:
  - 1.0 cortes de longitud 0.75
  - 1.0 cortes de longitud 2
Rollo maestro 17:
  - 1.0 cortes de longitud 0.75
  - 1.0 cortes de longitud 2
Rollo maestro 20:
  - 2.0 cortes de longitud 1.5
Rollo maestro 21:
  - 1.0 cortes de longitud 2
Rollo maestro 22:
  - 1.0 cortes de longitud 0.75
  - 1.0 cortes de longitu